<center>
    <h1>   
        ==#== Bankruptcy Predictive Model ==#==
    </h1>
</center>
 
<center><img src="https://cdn.corporatefinanceinstitute.com/assets/bankruptcy-1024x627.jpeg" alt="Bankruptcy" width="600" ></center>

<center> <h3> Please Upvote if you like the work 😊 </h3></center>

<center>Don't hesitate to give your comment on this notebook. It gives me motivation to improve the analysis in the future</center>

# Introduction
This program is used to find the right model for the bankruptcy model. 

# Contents
Contents:

* [1. Load Packages](#1)
* [2. Load Dataset](#2)
* [3. Drop NaN and duplicates values](#3)
* [4. Do Explanatory Data Analysis](#4)
* [5. Mapping Sentiment](#5) 
* [6. Modelling](#6)

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
# ref: #https://www.kaggle.com/jaimebecerraguerrero/simple-yet-powerful-bankrupt-prediction-model

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

<a id="1"></a>
    
<font size="+2" color="indigo"><b>1. Load Packages</b></font><br>

In [ ]:
# runtime
import timeit

# Data manipulation
import pandas as pd
import numpy as np

# Data visualization
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

# preprocessing
from sklearn.feature_selection import RFE
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler, Normalizer

# Ml model
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.preprocessing import StandardScaler

from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import SMOTE
from matplotlib import pyplot
from numpy import where

from sklearn.pipeline import Pipeline
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score

np.warnings.filterwarnings('ignore')

<a id="2"></a>
    
<font size="+2" color="indigo"><b>2. Load Dataset</b></font><br>


In [ ]:
data_df=pd.read_csv("/kaggle/input/company-bankruptcy-prediction/data.csv")

In [ ]:
data_df.head()

<a id="3"></a>
    
<font size="+2" color="indigo"><b>3. Find Null and Duplicate Values</b></font><br>


In [ ]:
display(data_df.isnull().sum())
print("====///====")
display(data_df.duplicated().sum())
print("====///====")
display(data_df.info())

> No null values detected

<a id="4"></a>
    
<font size="+2" color="indigo"><b>4. Feature Selection</b></font><br>

We have a problem regarding the large number of features to choose from. Jaime in his [notebook](https://www.kaggle.com/jaimebecerraguerrero/simple-yet-powerful-bankrupt-prediction-model) describes the feature selection using the random forest classifier which is great for reducing computational load.

Recursive feature elimination (RFE) is a feature selection process that suits a model and eliminates the weakest feature (or features) before the required number of features is achieved. Features are rated by the model's coef_ or feature importances_ attributes, and RFE aims to remove dependencies and collinearity by recursively deleting a small number of features per loop.

In [ ]:
# training set
X = data_df.iloc[:,1:].values
y = data_df.iloc[:,0].values.reshape(-1, 1)

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [ ]:
# determining optimal number of features
n_features = [5, 10, 15, 20, 25, 30, 35, 40]
for i in n_features:
    # Building the model based feature selection
    select = RFE(RandomForestClassifier(n_estimators=100, random_state=42), n_features_to_select=i)

    select.fit(X_train, y_train)

    mask = select.get_support()

    X_train_rfe = select.transform(X_train)
    X_test_rfe = select.transform(X_test)

    score = RandomForestClassifier().fit(X_train_rfe, y_train).score(X_test_rfe, y_test)
    
    print("Test score: {:.3f}".format(score), " number of features: {}".format(i))

In [ ]:
select = RFE(RandomForestClassifier(n_estimators=100, random_state=42), n_features_to_select=15)

select.fit(X_train, y_train)

mask = select.get_support()

mask

In [ ]:
X_train_rfe = select.transform(X_train)
X_test_rfe = select.transform(X_test)

score = RandomForestClassifier().fit(X_train_rfe, y_train).score(X_test_rfe, y_test)

print("Test score: {:.3f}".format(score), " number of features: {}".format(15))

features = pd.DataFrame({'features':list(data_df.iloc[:,1:].keys()), 'select':list(mask)})
features = list(features[features['select']==True]['features'])
features.append('Bankrupt?')

In [ ]:
data_df = data_df[features]


data_df.info()

<a id="5"></a>
    
<font size="+2" color="indigo"><b>5. Exploratory Data Analysis</b></font><br>

## STEP 1: UNIVARIATE ANALYSIS

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# Split feature as categorical and continuous features

cat_features=data_df.select_dtypes(exclude=np.number).columns
cont_features=data_df.select_dtypes(include=[np.number,'float64','int64']).columns

features=print(data_df.columns)

In [ ]:
# Exploring categorical data

plt.figure(figsize=(20,8))
for index, col in enumerate(cat_features, start=1):
    plt.subplot(4,4,index)
    plt.title(col)
    plt.pie(data_df[col].value_counts(), labels=data_df[col].value_counts().index)

In [ ]:
# Exploring continuous data

import matplotlib.pyplot as plt
plt.figure(figsize=(20,12))
for index, col in enumerate(cont_features, start=1):
    plt.subplot(4,5,index)
    plt.title(col)
    sns.distplot(data_df[col], kde_kws={'bw':0.5}) # bw is bandwith

In [ ]:
# Using Boxplot

import numpy as np
import seaborn as sns
cont_dummy=cont_features#.drop('store_and_fwd_flag')
plt.figure(figsize=(20,16))
for index, col in enumerate(cont_dummy, start=1):
    plt.subplot(4,5,index)
    plt.title(col)
    ax = sns.boxplot(data_df[col])


> We need to overcome the outliers problem

# STEP 2: BIVARIATE/MULTIVARIATE ANALYSIS

In [ ]:
corrmat = data_df.corr(method='pearson')
f, ax = plt.subplots(figsize=(12, 10))
sns.heatmap(corrmat, ax=ax, cmap="YlGnBu", linewidths=0.1, annot=True)

<a id="6"></a>
    
<font size="+2" color="indigo"><b>6. Handling Outliers</b></font><br>

In [ ]:
def cap_outliers(series, zscore_threshold=3, iqr_threshold=1.5, verbose=False, method="IQR"):
    if method=="IQR":
        #https://stackoverflow.com/questions/59489073/how-to-not-remove-but-handle-outliers-by-transforming-using-pandas
        '''Caps outliers to closest existing value within threshold (IQR).'''
        Q1 = series.quantile(0.25)
        Q3 = series.quantile(0.75)
        IQR = Q3 - Q1

        lbound = Q1 - iqr_threshold * IQR
        ubound = Q3 + iqr_threshold * IQR

        outliers = (series < lbound) | (series > ubound)

        series = series.copy()
        series.loc[series < lbound] = series.loc[~outliers].min()
        series.loc[series > ubound] = series.loc[~outliers].max()

        # For comparison purposes.
        if verbose:
                print('\n'.join(
                    ['Capping outliers by the IQR method:',
                     f'   IQR threshold: {iqr_threshold}',
                     f'   Lower bound: {lbound}',
                     f'   Upper bound: {ubound}\n']))
    elif method=="mean":
        '''Caps outliers to closest existing value within threshold (Modified Z-score).'''
        median_val = series.median()
        mad_val = series.mad() # Median absolute deviation

        z_score = (series - median_val) / mad_val
        outliers = abs(z_score) > zscore_threshold

        series = series.copy()
        series.loc[z_score > zscore_threshold] = series.loc[~outliers].max()
        series.loc[z_score < -zscore_threshold] = series.loc[~outliers].min() 

        # For comparison purposes.
        if verbose:
                lbound = median_val - zscore_threshold * mad_val
                ubound = median_val + zscore_threshold * mad_val
                print('\n'.join(
                    ['Capping outliers by the Modified Z-score method:',
                     f'   Z-score threshold: {zscore_threshold}',
                     f'   Lower bound: {lbound}',
                     f'   Upper bound: {ubound}\n']))
    elif method=="zscore":
        '''Caps outliers to closest existing value within threshold (Modified Z-score).'''
        median_val = series.median()
        mad_val = series.mad() # Median absolute deviation

        z_score = (series - median_val) / mad_val
        outliers = abs(z_score) > zscore_threshold

        series = series.copy()
        series.loc[z_score > zscore_threshold] = series.loc[~outliers].mean()
        series.loc[z_score < -zscore_threshold] = series.loc[~outliers].mean() 

        # For comparison purposes.
        if verbose:
                lbound = median_val - zscore_threshold * mad_val
                ubound = median_val + zscore_threshold * mad_val
                print('\n'.join(
                    ['Capping outliers by the Modified Z-score method:',
                     f'   Z-score threshold: {zscore_threshold}',
                     f'   Lower bound: {lbound}',
                     f'   Upper bound: {ubound}\n']))
    elif method=="median":
        '''Caps outliers to closest existing value within threshold (Modified Z-score).'''
        median_val = series.median()
        mad_val = series.mad() # Median absolute deviation

        z_score = (series - median_val) / mad_val
        outliers = abs(z_score) > zscore_threshold

        series = series.copy()
        series.loc[z_score > zscore_threshold] = series.loc[~outliers].max()
        series.loc[z_score < -zscore_threshold] = series.loc[~outliers].min() 

        # For comparison purposes.
        if verbose:
                lbound = median_val - zscore_threshold * mad_val
                ubound = median_val + zscore_threshold * mad_val
                print('\n'.join(
                    ['Capping outliers by the Modified Z-score method:',
                     f'   Z-score threshold: {zscore_threshold}',
                     f'   Lower bound: {lbound}',
                     f'   Upper bound: {ubound}\n']))
    elif method=="zscore":
        '''Caps outliers to closest existing value within threshold (Modified Z-score).'''
        median_val = series.median()
        mad_val = series.mad() # Median absolute deviation

        z_score = (series - median_val) / mad_val
        outliers = abs(z_score) > zscore_threshold

        series = series.copy()
        series.loc[z_score > zscore_threshold] = series.loc[~outliers].median()
        series.loc[z_score < -zscore_threshold] = series.loc[~outliers].median() 

        # For comparison purposes.
        if verbose:
                lbound = median_val - zscore_threshold * mad_val
                ubound = median_val + zscore_threshold * mad_val
                print('\n'.join(
                    ['Capping outliers by the Modified Z-score method:',
                     f'   Z-score threshold: {zscore_threshold}',
                     f'   Lower bound: {lbound}',
                     f'   Upper bound: {ubound}\n']))
    elif method=="log":
        #https://stackoverflow.com/questions/37890849/pandas-series-log-normalize
        series=series.map(lambda x: np.log(x))
        

    return series


remove_list = [' Net Value Growth Rate', 'Bankrupt?']   
features_treating = list(data_df.columns.values)

res = [i for i in features_treating if i not in remove_list]

features_treating = res
features_treating
for i, col in enumerate(features_treating):
    data_df[col]=cap_outliers(data_df[col],method="IQR", verbose=True)

In [ ]:
# Verification of Deleting Outliers

cont_dummy=cont_features#.drop('store_and_fwd_flag')
plt.figure(figsize=(20,16))
for index, col in enumerate(cont_dummy, start=1):
    plt.subplot(4,5,index)
    plt.title(col)
    ax = sns.boxplot(data_df[col])

<a id="7"></a>
    
<font size="+2" color="indigo"><b>7. Modelling</b></font><br>

In [ ]:
X=data_df.drop('Bankrupt?',axis=1) #axis=1 drop the bulk of column
y=data_df['Bankrupt?']

oversample = SMOTE()
X, y = oversample.fit_resample(X, y)

X_train, X_test, y_train, y_test=train_test_split(X,y,test_size=0.25,random_state=0)

#StSc = StandardScaler()
#X_train  = StSc.fit_transform(X_train)
#X_test  = StSc.fit_transform(X_test)

In [ ]:
for i, col in enumerate([XGBClassifier(eval_metric="logloss"),RandomForestClassifier(),LogisticRegression()]):
    print(col)
    pipeModel=Pipeline([('scaler', StandardScaler()), ('model', col)])
    
    # the benefit of using K-Fold is that we could calculate the cross validation value using some of the methods of scoring 
    kfold = KFold(n_splits=10, shuffle=True, random_state=0)
    cv_results = cross_val_score(pipeModel, X_train, y_train, cv=kfold)

    pipeModel.fit(X_train, y_train)
    # this is the scaled LR
    print('Score for',col,'method:', pipeModel.score(X_test, y_test))
    # the mean result (10 data) of negative mean squared error
    print('Score for',col,'method using cross_val_score:', cv_results.mean())
    y_pred=pipeModel.predict(X_test)
    #print(mean_squared_error(y_test, y_pred))
    print(classification_report(y_test,y_pred))
    roc_auc_score(y_test,y_pred)

In [ ]:
from sklearn.metrics import confusion_matrix
gbrt = GradientBoostingClassifier(n_estimators=100, random_state=42, max_depth=1).fit(X_train, y_train)
y_pred=gbrt.predict(X_test)
print("training set score : {:.2f}".format(gbrt.score(X_train, y_train)))
print("test set score: {:.2f}".format(gbrt.score(X_test, y_test)))

confusion_matrix(y_test, y_pred)


<a id="8"></a>
    
<font size="+2" color="indigo"><b>8. Conclusion</b></font><br>

> We have trained a model with training set score 90% and test set score 90% using Gradient Boosting Classifier. Also, we have removed outliers using Interquartiles Caping Outliers Methods. 

<center> <h3> Please Upvote if you like the work 😊 </h3></center>
<center>Don't hesitate to give your comment on this notebook. It gives me motivation to improve the analysis in the future</center>